# Toronto Neighbourhoods

This notebook is used for data science capstone project.
Before we get the data and start exploring it, let's download all the dependencies that we will need.

In [149]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis

import folium # map rendering library

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# import k-means from clustering stage
from sklearn.cluster import KMeans

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

print('Libraries imported.')

Libraries imported.


Getting the list of toronto neighbourhoods by web scrapping

In [150]:
# results = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M')
# results
dfs = pd.read_html('https://en.wikipedia.org/w/index.php?title=List_of_postal_codes_of_Canada:_M&oldid=1011037969')
print(len(dfs))

3


So there are three dataframes, but it's the first one which we need.

In [151]:
df= dfs[0]


Now, we have to clean the data, so first removing space from column name to correct the naming conventions

In [152]:
df.rename(columns={'Postal Code': 'PostalCode'},inplace=True)

Removing all the rows where Borough and neighbourhood is not assigned.

In [153]:
df = df.query('(Borough != "Not assigned") and (Neighbourhood != "Not assigned")')

In [154]:
df.head()

,PostalCode,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
5,M6A,North York,"Lawrence Manor, Lawrence Heights"
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


Checking if any row has neighbourhood which is not assigned.

In [155]:
df.query('(Neighbourhood == "Not assigned")')

,PostalCode,Borough,Neighbourhood


In [156]:
df.shape

(103, 3)

Now getting the coordinates of neighbourhoods from a file provided by this course

In [157]:
data= pd.read_csv('data/Geospatial_Coordinates.csv')
data.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [158]:
data.shape

(103, 3)

In [159]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

Now joining the two dataframes

In [160]:
merged = pd.merge(left=df, right=data,how='left', left_on='Postal Code', right_on='Postal Code')
merged.head()

KeyError: 'Postal Code'

## Now keeping only toronto boroughs

In [21]:
toronto_df = merged.query('(Borough.str.contains("Toronto")) or (Borough.str.contains("toronto"))')
toronto_df.reset_index(drop=True,inplace=True)
toronto_df.head()

,Postal Code,Borough,Neighbourhood,Latitude,Longitude
0,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
1,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494
2,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937
3,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418
4,M4E,East Toronto,The Beaches,43.676357,-79.293031


## Creating a map of Toronto with neighborhoods superimposed on top.

In [22]:
# create map of Toronto using latitude and longitude values
latitude = 43.7253427
longitude = -79.5208805
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, borough, neighborhood in zip(toronto_df['Latitude'], toronto_df['Longitude'], toronto_df['Borough'], toronto_df['Neighbourhood']):
    label = '{}, {}'.format(neighborhood, borough)
    # iframe = folium.html(label)
    iframe = folium.IFrame(html=label, width=250, height=75)
    label = folium.Popup(iframe, min_width= 1000, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=True).add_to(map_toronto)  
    
map_toronto

Next, we are going to start utilizing the Foursquare API to explore the neighborhoods and segment them.

## Defining Foursquare Credentials and Version

In [23]:
CLIENT_ID = 'FJBBHL0CQFIZKY0C3JXRMJ1Q3YYQRG0W5OGAREGYE1TMYRL3' # your Foursquare ID
CLIENT_SECRET = 'CM0T2DDI4IXNZAAVZNHW2Z1JQ1QYQQXUYF55GTMH4OF3BY0B' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version
LIMIT = 100 # A default Foursquare API limit value

print('credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

credentails:
CLIENT_ID: FJBBHL0CQFIZKY0C3JXRMJ1Q3YYQRG0W5OGAREGYE1TMYRL3
CLIENT_SECRET:CM0T2DDI4IXNZAAVZNHW2Z1JQ1QYQQXUYF55GTMH4OF3BY0B


## Let's explore the first neighborhood in our dataframe.


In [24]:
toronto_df.loc[0, 'Neighbourhood']

'Regent Park, Harbourfront'

In [25]:
neighbourhood_latitude = toronto_df.loc[0, 'Latitude'] # neighborhood latitude value
neighbourhood_longitude = toronto_df.loc[0, 'Longitude'] # neighborhood longitude value

neighbourhood_name = toronto_df.loc[0, 'Neighbourhood'] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(neighbourhood_name, 
                                                               neighbourhood_latitude, 
                                                               neighbourhood_longitude))

Latitude and longitude values of Regent Park, Harbourfront are 43.6542599, -79.3606359.


#### First, let's create the GET request URL

In [27]:
LIMIT = 100 # limit of number of venues returned by Foursquare API

radius = 500 # define radius

url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighbourhood_latitude, 
    neighbourhood_longitude, 
    radius, 
    LIMIT)
url # display URL

'https://api.foursquare.com/v2/venues/explore?&client_id=FJBBHL0CQFIZKY0C3JXRMJ1Q3YYQRG0W5OGAREGYE1TMYRL3&client_secret=CM0T2DDI4IXNZAAVZNHW2Z1JQ1QYQQXUYF55GTMH4OF3BY0B&v=20180605&ll=43.6542599,-79.3606359&radius=500&limit=100'

In [28]:
results = requests.get(url).json()

Creating function that extracts the category of the venue

In [29]:
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

Now we are ready to clean the json and structure it into a _pandas_ dataframe.

In [30]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = pd.json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

,name,categories,lat,lng
0,Roselle Desserts,Bakery,43.653447,-79.362017
1,Tandem Coffee,Coffee Shop,43.653559,-79.361809
2,Cooper Koo Family YMCA,Distribution Center,43.653249,-79.358008
3,Body Blitz Spa East,Spa,43.654735,-79.359874
4,Impact Kitchen,Restaurant,43.656369,-79.356980


In [31]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

46 venues were returned by Foursquare.


#### Creating the function to repeat the same process to all the neighborhoods of Toronto

In [32]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighbourhood', 
                  'Neighbourhood Latitude', 
                  'Neighbourhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [36]:
toronto_venues = getNearbyVenues(names=toronto_df['Neighbourhood'],
                                   latitudes=toronto_df['Latitude'],
                                   longitudes=toronto_df['Longitude']
                                  )

Regent Park, Harbourfront
Queen's Park, Ontario Provincial Government
Garden District, Ryerson
St. James Town
The Beaches
Berczy Park
Central Bay Street
Christie
Richmond, Adelaide, King
Dufferin, Dovercourt Village
Harbourfront East, Union Station, Toronto Islands
Little Portugal, Trinity
The Danforth West, Riverdale
Toronto Dominion Centre, Design Exchange
Brockton, Parkdale Village, Exhibition Place
India Bazaar, The Beaches West
Commerce Court, Victoria Hotel
Studio District
Lawrence Park
Roselawn
Runnymede, The Junction, Weston-Pellam Park, Carlton Village
Davisville North
Forest Hill North & West, Forest Hill Road Park
High Park, The Junction South
North Toronto West, Lawrence Park
The Annex, North Midtown, Yorkville
Parkdale, Roncesvalles
Davisville
University of Toronto, Harbord
Runnymede, Swansea
Moore Park, Summerhill East
Kensington Market, Chinatown, Grange Park
Summerhill West, Rathnelly, South Hill, Forest Hill SE, Deer Park
CN Tower, King and Spadina, Railway Lands, Harb

In [37]:
print(toronto_venues.shape)
toronto_venues.head()

(1602, 7)


,Neighbourhood,Neighbourhood Latitude,Neighbourhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"Regent Park, Harbourfront",43.65426,-79.360636,Roselle Desserts,43.653447,-79.362017,Bakery
1,"Regent Park, Harbourfront",43.65426,-79.360636,Tandem Coffee,43.653559,-79.361809,Coffee Shop
2,"Regent Park, Harbourfront",43.65426,-79.360636,Cooper Koo Family YMCA,43.653249,-79.358008,Distribution Center
3,"Regent Park, Harbourfront",43.65426,-79.360636,Body Blitz Spa East,43.654735,-79.359874,Spa
4,"Regent Park, Harbourfront",43.65426,-79.360636,Impact Kitchen,43.656369,-79.356980,Restaurant


Let's check how many venues were returned for each neighborhood

In [39]:
toronto_venues.groupby('Neighbourhood').count()

,Neighbourhood Latitude,Neighbourhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighbourhood,,,,,,
Berczy Park,59,59,59,59,59,59
"Brockton, Parkdale Village, Exhibition Place",23,23,23,23,23,23
"Business reply mail Processing Centre, South Central Letter Processing Plant Toronto",16,16,16,16,16,16
"CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport",15,15,15,15,15,15
Central Bay Street,61,61,61,61,61,61
Christie,16,16,16,16,16,16
Church and Wellesley,73,73,73,73,73,73
"Commerce Court, Victoria Hotel",100,100,100,100,100,100
Davisville,35,35,35,35,35,35


Let's find out how many unique categories can be curated from all the returned venues


In [40]:
print('There are {} uniques categories.'.format(len(toronto_venues['Venue Category'].unique())))

There are 233 uniques categories.


## Finding the top 10 categories in toronto.

In [148]:
top10Venues = toronto_venues['Venue Category'].value_counts()[:10].index.tolist()
top10Venues

['Coffee Shop',
 'Café',
 'Restaurant',
 'Italian Restaurant',
 'Hotel',
 'Park',
 'Japanese Restaurant',
 'Bakery',
 'Sushi Restaurant',
 'Gym']

## Analyzing each neighbourhood

In [132]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Neighbourhood'] = toronto_venues['Neighbourhood'] 

# move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

toronto_onehot.head()

,Neighbourhood,Airport,Airport Food Court,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Workshop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Stadium,Basketball Stadium,Beach,Bed & Breakfast,Beer Bar,Beer Store,Belgian Restaurant,Bistro,Board Shop,Boat or Ferry,Bookstore,Brazilian Restaurant,Breakfast Spot,Brewery,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Butcher,Café,Cajun / Creole Restaurant,Candy Store,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Auditorium,College Cafeteria,College Gym,College Rec Center,Colombian Restaurant,Comfort Food Restaurant,Comic Shop,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Escape Room,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,General Travel,German Restaurant,Gift Shop,Gluten-free Restaurant,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Harbor / Marina,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Home Service,Hookah Bar,Hospital,Hostel,Hotel,Hotel Bar,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Korean Restaurant,Lake,Latin American Restaurant,Light Rail Station,Lingerie Store,Liquor Store,Lounge,Market,Martial Arts School,Massage Studio,Mediterranean Restaurant,Men's Store,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Moroccan Restaurant,Movie Theater,Moving Target,Museum,Music Venue,Neighborhood,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Park,Performing Arts Venue,Pet Store,Pharmacy,Pizza Place,Plane,Playground,Plaza,Poke Place,Pool,Portuguese Restaurant,Poutine Place,Pub,Ramen Restaurant,Record Shop,Rental Car Location,Restaurant,Roof Deck,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Skate Park,Skating Rink,Smoke Shop,Smoothie Shop,Snack Place,Soup Place,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Stadium,Stationery Store,Steakhouse,Strip Club,Supermarket,Sushi Restaurant,Swim School,Taco Place,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tea Room,Thai Restaurant,Theater,Theme Restaurant,Thrift / Vintage Store,Tibetan Restaurant,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Yoga Studio
0,"Regent Park, Harbourfront",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,"Regent Park, Harbourfront",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0

In [133]:
toronto_onehot.shape

(1602, 234)

#### Next, let's group rows by neighbourhood and by taking the mean of the frequency of occurrence of each category


In [134]:
toronto_grouped = toronto_onehot.groupby('Neighbourhood').mean().reset_index()
toronto_grouped.shape

(39, 234)

## Finally, finding least competitive Neighbourhood

For example if you want to open a coffee shop you can consider following neighbourhoods, because these neighbourhoods don't have any coffee shop.

In [142]:
temp = toronto_grouped[toronto_grouped['Coffee Shop'] == 0.0]
temp['Neighbourhood'].tolist()

['Business reply mail Processing Centre, South Central Letter Processing Plant Toronto',
 'Davisville North',
 'Dufferin, Dovercourt Village',
 'Forest Hill North & West, Forest Hill Road Park',
 'High Park, The Junction South',
 'India Bazaar, The Beaches West',
 'Lawrence Park',
 'Rosedale',
 'Roselawn',
 'Runnymede, The Junction, Weston-Pellam Park, Carlton Village',
 'The Beaches']

### Now showing less competitive neighbourhoods for each venue type

In [143]:
for venue in top10Venues:
    print("----" + venue + "----")
    temp = toronto_grouped[toronto_grouped[venue] == 0.0]
    print(temp['Neighbourhood'])

----Coffee Shop----
2     Business reply mail Processing Centre, South C...
9                                      Davisville North
10                         Dufferin, Dovercourt Village
12      Forest Hill North & West, Forest Hill Road Park
15                        High Park, The Junction South
16                       India Bazaar, The Beaches West
18                                        Lawrence Park
25                                             Rosedale
26                                             Roselawn
28    Runnymede, The Junction, Weston-Pellam Park, C...
35                                          The Beaches
Name: Neighbourhood, dtype: object
----Café----
2     Business reply mail Processing Centre, South C...
3     CN Tower, King and Spadina, Railway Lands, Har...
9                                      Davisville North
12      Forest Hill North & West, Forest Hill Road Park
16                       India Bazaar, The Beaches West
18                                  

### Finding less competitive neighbourhoods overall.

First, filtering out other Venue types and only keeping top 10 types

In [135]:
top10Venues.append('Neighbourhood')
toronto_grouped.drop(toronto_grouped.columns.difference(top10Venues), 1, inplace=True)
top10Venues.remove('Neighbourhood')
toronto_grouped.head()

,Neighbourhood,Bakery,Café,Coffee Shop,Gym,Hotel,Italian Restaurant,Japanese Restaurant,Park,Restaurant,Sushi Restaurant
0,Berczy Park,0.050847,0.016949,0.084746,0.000000,0.016949,0.000000,0.016949,0.016949,0.033898,0.016949
1,"Brockton, Parkdale Village, Exhibition Place",0.043478,0.130435,0.086957,0.043478,0.000000,0.043478,0.000000,0.000000,0.043478,0.000000
2,"Business reply mail Processing Centre, South C...",0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.062500,0.062500,0.000000
3,"CN Tower, King and Spadina, Railway Lands, Har...",0.000000,0.000000,0.066667,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
4,Central Bay Street,0.000000,0.049180,0.180328,0.000000,0.000000,0.049180,0.032787,0.016393,0.016393,0.016393


Then, caluclate the mean of each neighbourhood.

In [162]:
toronto_grouped['mean'] = toronto_grouped.mean(axis=1)
toronto_grouped

,Neighbourhood,Bakery,Café,Coffee Shop,Gym,Hotel,Italian Restaurant,Japanese Restaurant,Park,Restaurant,Sushi Restaurant,mean
0,Berczy Park,0.050847,0.016949,0.084746,0.000000,0.016949,0.000000,0.016949,0.016949,0.033898,0.016949,0.025424
1,"Brockton, Parkdale Village, Exhibition Place",0.043478,0.130435,0.086957,0.043478,0.000000,0.043478,0.000000,0.000000,0.043478,0.000000,0.039130
2,"Business reply mail Processing Centre, South C...",0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.062500,0.062500,0.000000,0.012500
3,"CN Tower, King and Spadina, Railway Lands, Har...",0.000000,0.000000,0.066667,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.006667
4,Central Bay Street,0.000000,0.049180,0.180328,0.000000,0.000000,0.049180,0.032787,0.016393,0.016393,0.016393,0.036066
5,Christie,0.000000,0.187500,0.062500,0.000000,0.000000,0.062500,0.000000,0.125000,0.062500,0.000000,0.050000
6,Church and Wellesley,0.000000,0.027397,0.082192,0.000000,0.027397,0.013699,0.068493,0.013699,0.041096,0.068493,0.034247
7,"Commerce Court, Victoria Hotel",0.020000,0.060000,0.130000,0.040000,0.050000,0.040000,0.030000,0.010000,0.070000,0.000000,0.045000
8,Davisville,0.000000,0.057143,0.057143,0.057143,0.000000,0.057143,0.000000,0.028571,0.028571,0.057143,0.034286
9,Davisville North,0.000000,0.000000,0.000000,0.000000,0.142857,0.000000,0.000000,0.142857,0.000000,0.000000,0.028571


Low mean value indicates that it has less number of our top 10 venues.

In [170]:
top5= toronto_grouped.sort_values('mean', ascending=True).head().reset_index(drop=True)

### So, following are the top 5 neighbourhoods with less competition overall and any thing you open from that top 10 list would have higher chance of success.

In [178]:

for index, row in top5.iterrows():
    
    print(str(index+ 1) + ". " + row['Neighbourhood'])

1. The Beaches
2. Runnymede, The Junction, Weston-Pellam Park, Carlton Village
3. Roselawn
4. CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport
5. Business reply mail Processing Centre, South Central Letter Processing Plant Toronto
